The theory for this notebook is developed in `ZonotopesNonlinearReach`.

In [1]:
# load packages
using Plots
using LazySets, MathematicalSystems, Reachability
using LazySets.Approximations
using Reachability: center
using Reachability.ReachSets: Φ₁
using IntervalArithmetic, ValidatedNumerics
using LazySets: Interval, translate
using TaylorSeries
using TaylorSeries: gradient, jacobian, hessian, derivative
const ∂ = derivative
using RecursiveArrayTools

┌ Info: Recompiling stale cache file /Users/forets/.julia/compiled/v1.1/Reachability/CHV4V.ji for Reachability [774a0091-654f-5c65-bbdc-ad5b67b45832]
└ @ Base loading.jl:1184


After loading the packages, you should precompile the functions under the section *auxiliary functions* below.

In [2]:
function vanderpol()
    # number of Taylor terms considered in the linearization
    taylor_terms = 4

    # define the working variables and fix the max order in the Taylor series expansion
    x = set_variables("x", numvars=2, order=taylor_terms)

    # define the ODE
    f = Vector{TaylorN{Float64}}(undef, 2)
    f[1] = x[2]
    f[2] = x[2] * (1-x[1]^2) - x[1]

    # define the initial-value problem
    𝑋₀ = Hyperrectangle(low=[1.25, 2.35], high=[1.55, 2.45])
    
    #𝑋₀ = convert(Zonotope, 𝑋₀)
    #𝑋₀ = split(𝑋₀, [10, 10])[1] # temp
    
    𝑆 = BlackBoxContinuousSystem(f, 2)
    𝑃 = InitialValueProblem(𝑆, 𝑋₀)

    # general options
    𝑂 = Options(:T => 5.0, :plot_vars=>[1, 2])

    # algorithm-specific options
    𝑂_ASB08 = Options(:δ => 0.01, :θ=>fill(0.05, 2), :taylor_terms=>taylor_terms,
                      :split_blocks=>[2, 2], :max_chunks=>5000, :opC => BFFPSV18(:δ => 0.01/10))

    return 𝑃, 𝑂, 𝑂_ASB08 
end

vanderpol (generic function with 1 method)

In [3]:
@time begin 𝑃, 𝑂, 𝑂_ASB08 = vanderpol() end;

  3.468925 seconds (6.68 M allocations: 332.380 MiB, 13.84% gc time)


In [4]:
function laubloomis(; T=20.0, W=0.01, plot_vars=[0, 4],
                      property=(t,x)->x[4] < 4.5,
                      project_reachset=true)

    # number of Taylor terms considered in the linearization
    taylor_terms = 4

    # define the working variables and fix the max order in the Taylor series expansion
    x = set_variables("x", numvars=7, order=taylor_terms)

    # equations, x' = f(x(t))
    f = Vector{TaylorN{Float64}}(undef, 7)
    f[1] = 1.4*x[3] - 0.9*x[1]
    f[2] = 2.5*x[5] - 1.5*x[2]
    f[3] = 0.6*x[7] - 0.8*(x[2]*x[3])
    f[4] = 2 - 1.3*(x[3]*x[4])
    f[5] = 0.7*x[1] - (x[4]*x[5])
    f[6] = 0.3*x[1] - 3.1*x[6]
    f[7] = 1.8*x[6] - 1.6*(x[2]*x[7])

    𝐹 = BlackBoxContinuousSystem(f, 7)

    X0c = [1.2, 1.05, 1.5, 2.4, 1.0, 0.1, 0.45]
    X0 = Hyperrectangle(X0c, fill(W, 7))

    # instantiate the IVP
    𝑃 = InitialValueProblem(𝐹, X0)

    # general options
    𝑂 = Options(:T=>T, :plot_vars=>plot_vars, :property=>property,
                :project_reachset=>project_reachset, :mode=>"check")

    # algorithm-specific options
    𝑂_ASB08 = Options(:δ => 0.1, :θ=>fill(0.05, 7), :taylor_terms=>taylor_terms,
                      :split_blocks=>[2, 2, 2, 2, 2, 2, 2], :max_chunks=>5000, :opC=>BFFPSV18(:δ => 0.01))

    return 𝑃, 𝑂, 𝑂_ASB08     
    return (𝑃, 𝑂, 𝑂_ASB08)
end

laubloomis (generic function with 1 method)

In [5]:
@time begin 𝑃, 𝑂, 𝑂_ASB08 = laubloomis() end;

  0.574876 seconds (879.88 k allocations: 47.046 MiB, 31.38% gc time)


### Algorithm

In [126]:
MathematicalSystems.AbstractContinuousSystem

AbstractContinuousSystem

In [129]:
using MathematicalSystems: AbstractContinuousSystem

const HRECT = ReachSet{Hyperrectangle{Float64}, Float64}
const ZONO = ReachSet{Zonotope{Float64}, Float64}

# TODO: generalize to the set type in the options 
_opC_return_type(opC::BFFPSV18) = Vector{HRECT}
_opC_return_type(opC::GLGM06) = Vector{ZONO}

struct LinearizedSystem{PT, VT} <: AbstractContinuousSystem
    𝑃::PT # linearized IVP
    x̃::VT # lineariztion point
    I::IntervalArithmetic.Interval{Float64} # domain of validity of the linearization
end

In [118]:


# The structure that holds the flowpipe, Rsets, is a VectorOfArray
function post(𝑃::InitialValueProblem{<:BBCS}, 𝑂general::Options, 𝑂::Options)

    𝑂 = merge(𝑂, 𝑂general)

    # get nonlinear ODE
    f = 𝑃.s.f
    
    # unrwap commonly used options
    T, δ, δcont, θ = 𝑂[:T], 𝑂[:δ], 𝑂[:opC].options[:δ], 𝑂[:θ]
    split_blocks, max_chunks = 𝑂[:split_blocks], 𝑂[:max_chunks]
    
    # final result is stored in Rsets
    T = _opC_return_type(𝑂[:opC])
    Rsets = VectorOfArray(Vector{Vector{T}}(), (1, 1))
    
    # count number of continuous post chunks added to Rsets
    nchunks = 0

    𝑃_queue = Vector{typeof(𝑃)}()
    push!(𝑃_queue, 𝑃)

    # integer for the chunk being computed: [δ*(k-1), δk]
    # ie. k = 0 => [0, δ]
    #     k = 1 => [δ, 2δ]
    #          . . .
    #     k = N => [δ(N-1), δN]
    k_queue = Vector{Int}()
    push!(k_queue, 1)

    
    while !isempty(𝑃_queue)
        println("length(𝑃_queue) = $(length(𝑃_queue))")
        𝑃_curr = pop!(𝑃_queue)
        k_curr = pop!(k_queue)

        # obtain linearized system
        Lin = linearize(𝑃_curr, δ)
        Plin = Lin.P
        
        # compute flowpipe of the linearized system
        Rlin = solve(Plinear.P, 𝑂cont, op=𝑂[:opC])

        # compute set of admissible linearization errors
        R̄err, L̄ = admissible_error(𝑃lin.s.A, δ, θ, n=2)
        L = lagrange_remainder(f, Rlin, R̄err, x̃, n=2)

         if !(L ⊆ L̄)
             println("splitting chunk $nchunks")
             𝑋₀split = split(𝑃lin.x0, split_blocks)

             for 𝑋₀i in 𝑋₀split
                 push!(𝑃_queue, IVP(𝑃.s, 𝑋₀i))
                 push!(k_queue, k_curr)
             end
         else
             nchunks += 1
             println("adding chunk $nchunks")
             _add_chunk!(Rsets[k_curr], Rlin, R̄err, δ * (k_curr - 1) )
             if δ * (k_curr + 1) < T
                push!(𝑃_queue, IVP(𝑃.s, Rsets[k_curr][end].X))
                push!(k_queue, k_curr + 1)
             end
         end
            
         if nchunks > max_chunks
             error("maximum number of chunks exceeded")
             #return ReachSolution(Rsets, 𝑂)
         end
            
        # inclusion test
        # . . . 
    end 

    println("total chunks = $nchunks")
    return Rsets
    #return ReachSolution(Rsets, 𝑂)
end

post (generic function with 1 method)

In [119]:
T = _opC_return_type(BFFPSV18())
Rsets = VectorOfArray(Vector{Vector{T}}(), (1, 1))
push!(Rsets, Vector{HRECT}())

1-element Array{Array{Array{ReachSet{Hyperrectangle{Float64},Float64},1},1},1}:
 []

In [120]:
typeof(Rsets)

VectorOfArray{Array{ReachSet{Hyperrectangle{Float64},Float64},1},2,Array{Array{Array{ReachSet{Hyperrectangle{Float64},Float64},1},1},1}}

In [121]:
typeof(Rsets[1])

Array{Array{ReachSet{Hyperrectangle{Float64},Float64},1},1}

In [94]:
@time Rsets = post(𝑃, 𝑂, 𝑂_ASB08);
#nsets = length(Rsol.Xk)

length(𝑃_queue) = 1


UndefVarError: UndefVarError: linearize not defined

In [9]:
maximum([Rsol.Xk[i].t_end for i in 1:nsets])

UndefVarError: UndefVarError: nsets not defined

In [ ]:
plot(Rsol, xlab="x", ylab="y", alpha=.5, indices=1:500:nsets)
plot!(x->x, x->2.75, -3., 3., line=2, color="red", linestyle=:dash, legend=nothing)

In [ ]:
# check that specification holds
@assert all([ρ([0.0, 1.0], Rsol.Xk[i].X) < 2.75 for i in eachindex(Rsol.Xk)])

In [ ]:
maximum([ρ([0.0, 1.0], Rsol.Xk[i].X) for i in eachindex(Rsol.Xk)])

In [ ]:
any([Rsol.Xk[i].X ⊆ Rsol.Xk[1].X for i in 2:nsets])

### Appendix: auxiliary functions

In [ ]:
function linearize(𝑃::IVP{<:BBCS}, δ)

    # nonlinear ODE
    f = 𝑃.s.f
    
    # initial set of current chunk
    𝑋₀i = 𝑃.x0

    # linearization point for current chunk
    c = center(𝑋₀i)
    x̃ = c + δ/2 * f(c)

    # evaluate Jacobian at the linearization point
    Ax̃ = jacobian(f, x̃) #  map(x -> evaluate(x, x̃), Jf)
    bx̃ = f(x̃) - Ax̃ * x̃

    # instantiate linearized system; it doesn't have state constraints
    𝑆lin = ConstrainedAffineContinuousSystem(Ax̃, bx̃, Universe(2));
    𝑃lin = IVP(𝑆lin, 𝑋₀i)
    return x̃, 𝑃lin
end

In [95]:
function _add_chunk!(Rsets::Vector{HRECT}, Rlin, R̄err, t0)
    @inbounds for i in eachindex(Rlin.Xk)
        Ri = Rlin.Xk[i].X ⊕ R̄err
        Ri = overapproximate(Ri, Hyperrectangle)
        Ri = ReachSet(Ri, t0 + Rlin.Xk[i].t_start, t0 + Rlin.Xk[i].t_end)
        push!(Rsets[k], Ri)
    end
    return Rsets
end

function _add_chunk!(Rsets::Vector{ZONO}, k, Rlin, R̄err, t0)
    #println("using zonotopes")
    @inbounds for i in eachindex(Rlin.Xk)
        Ri = minkowski_sum(Rlin.Xk[i].X, convert(Zonotope, R̄err))
        Ri = ReachSet(Ri, t0 + Rlin.Xk[i].t_start, t0 + Rlin.Xk[i].t_end)
        push!(Rsets[k], Ri)
    end
    return Rsets
end

_add_chunk! (generic function with 4 methods)

In [96]:
function admissible_error(Ax̃, δ, θ; n=2)
    @assert n == 2
    Φ₁_Aδ = Φ₁(Ax̃, δ)
    R̄err = Hyperrectangle(zeros(2), θ*δ)
    l̄ = abs.(inv(Φ₁_Aδ)) * θ * δ
    L̄ = Hyperrectangle(zeros(2), l̄)
    return R̄err, L̄
end

admissible_error (generic function with 1 method)

In [ ]:
function lagrange_remainder(f, Rlin, R̄err, x̃; n=2)
    @assert n == 2
    
    Hf₁ = [∂(f[1], (2, 0)) ∂(f[1], (1, 1));
           ∂(f[1], (1, 1)) ∂(f[1], (0, 2))]
    Hf₂ = [∂(f[2], (2, 0)) ∂(f[2], (1, 1));
           ∂(f[2], (1, 1)) ∂(f[2], (0, 2))]

    R̂lin = ConvexHullArray([Ri.X for Ri in Rlin.Xk]) ⊕ R̄err
    R̂lin_rect = overapproximate(R̂lin, Hyperrectangle)

    ξ = CH(Singleton(x̃), R̂lin_rect)
    ξ_rect = overapproximate(ξ, Hyperrectangle)
    ξ_box = convert(IntervalBox, ξ_rect)

    Hf₁_box = map(Hf_ij -> evaluate(Hf_ij, ξ_box), Hf₁)
    Hf₂_box = map(Hf_ij -> evaluate(Hf_ij, ξ_box), Hf₂)

    R̂lin_zono = convert(Zonotope, R̂lin_rect)

    γ = abs.(R̂lin_zono.center - x̃) + sum(abs.(R̂lin_zono.generators), dims=2)
    
    G = [sup.(abs.(Hf₁_box)), sup.(abs.(Hf₂_box))];
    li_zono = [(1/2 * transpose(γ) * G[i] * γ)[1, 1] for i in 1:n]
    L = Hyperrectangle(zeros(n), li_zono)
    return L
end

### Other scripts

In [ ]:
#=
@time begin
# number of Taylor terms considered in the linearization
taylor_terms = 4

# define the working variables and fix the max order in the Taylor series expansion
x = set_variables("x", numvars=2, order=taylor_terms)

# define the ODE
f = Vector{TaylorN{Float64}}(undef, 2)
f[1] = x[2]
f[2] = x[2] * (1-x[1]^2) - x[1]

# define the initial-value problem
𝑋₀ = Hyperrectangle(low=[1.25, 2.35], high=[1.55, 2.45])
𝑆 = BlackBoxContinuousSystem(f, 2)
𝑃 = InitialValueProblem(𝑆, 𝑋₀)

# take the gradient of the vector field symbolically
#∇f = [gradient(f[i]) for i in 1:2]

# take the Jacobian matrix of the vector field symbolically
#Jf = [∂(f[1], (1, 0)) ∂(f[1], (0, 1));
#      ∂(f[2], (1, 0)) ∂(f[2], (0, 1))]

# algorithm-specific options
𝑂 = Options(:δ => 0.02, :δcont => 0.02/10, :max_order=>2, :θ=>fill(0.05, 2))

# unwrap options
δ = O[:δ]
θ = O[:θ]

# collection of flowpipes; the set type depends on add_chunk! and the continuous post
Rsets = Vector{ReachSet{Hyperrectangle{Float64}, Float64}}()
end
=#

In [ ]:
#=
@time begin
x̃, 𝑃lin = linearize(𝑃, δ)
end;
𝑋₀i = 𝑃lin.x0

# use zonotope-based continuous reach
@time begin
Rlin_zono = solve(𝑃lin, Options(:T=>O[:δ]), op=GLGM06(:δ=>O[:δcont], :max_order=>O[:max_order]))
end;

# use decomposition-based continuous reach
@time begin
Rlin_box = solve(𝑃lin, Options(:T=>O[:δ]), op=BFFPSV18(:δ=>O[:δcont]))
end;

plot(Rlin_zono, alpha=.5)
plot!(Rlin_box, alpha=.5, xlab="x", ylab="y")

Rlin = Rlin_zono  # decide which continuoust post to use

@time begin
R̄err, L̄ = admissible_error(𝑃lin.s.A, δ, θ; n=2)
L = lagrange_remainder(f, Rlin, R̄err, x̃; n=2)
end
=#